In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import random 
#from sklearn import datasets
#iris = datasets.load_iris()

In [18]:
def getData(df):
    X = df.iloc[:,:-1]#.values
    y = df.iloc[:,-1]#.values
    return X, y


def add_random_noise(y,noise_level = 0.2):

    labels = list(pd.unique(y))
    n = len(y)
    n_noisy = round(n * noise_level)
    idx_noisy = random.sample(range(n),n_noisy)
    y_noisy = y.copy()
    for idx in idx_noisy:
        clean_label = y[idx]
        labels = list(pd.unique(y))
        labels.remove(clean_label)
        noisy_label = random.choice(labels)
        y_noisy.iloc[idx] = noisy_label

    return y_noisy

In [45]:
# original filter https://rdrr.io/cran/NoiseFiltersR/src/R/ENN.R
def ENN(df, n_neighbors = 5):
    """
    A filter that finds n neibhours for each datapoint and drops the datapoint (FALSE in the clean_list) 
    if the majority of the neibhors has a different label
    Returns a boolean list with TRUE for clean and FALSE for noisy data point
    """
    X, y = getData(df)
    N = len(df)
    clean_list = []

    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X.values, y.values)
    for i in range(N):
        own_label = y[i]
        print(own_label)
        neighbors_idx = knn.kneighbors(X.iloc[i,:].values.reshape(1, -1), return_distance=False)
        neighbor_labels = y.values[neighbors_idx][0]
        print(neighbor_labels)
        values, counts = np.unique(neighbor_labels, return_counts=True)
        most_frequent_label = values[counts == counts.max()]
        clean_list.append([most_frequent_label == own_label][0][0])

    #clean_samples =  df[clean_list]
    #noisy_samples = df[np.invert(clean_list)]
    return clean_list





In [46]:
# Load the iris dataset into a Pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

X, y = getData(df)
y_noisy = add_random_noise(y,noise_level = 0.2)

df_noisy = df.copy()
df_noisy.iloc[:,-1] = y_noisy

clean_list = ENN(df_noisy, n_neighbors = 5)

noisy = y == y_noisy
acc = (noisy == clean_list).sum()/len(clean_list)
print('The ENN filter has accuracy of {}'.format(round(acc,3)))

setosa
['setosa' 'setosa' 'setosa' 'setosa' 'setosa']
versicolor
['versicolor' 'setosa' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'setosa' 'setosa']
versicolor
['versicolor' 'versicolor' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'virginica' 'virginica']
setosa
['setosa' 'setosa' 'virginica' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'virginica' 'setosa']
setosa
['setosa' 'setosa' 'versicolor' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'setosa' 'setosa' 'setosa']
virginica
['virginica' 'setosa' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'versicolor' 'setosa' 'setosa' 'setosa']
setosa
['setosa' 'setosa' 'virginica' 'setosa' 'setosa']
setosa
['setosa' 'virginica' 'setosa' 'setosa' 'versicolor']
setosa
['setosa' 'virginica' 'setosa' 'setosa' 'versicolor']
setosa
['setosa' 'setosa' 'setosa' 'virginica' 'setosa']
setosa
['setosa'

In [7]:
import sys
sys.path.append('..')
import ENN

In [1]:
#import sys
#sys.path.append('.')
#import Filter


import pandas as pd
import numpy as np

import sys
sys.path.append('.')
import Filter

class ENN(Filter):

   # X: FArray
   # y: FArray
    #df: pandas.core.frame.DataFrame
    #n_neighbors: int

    def __init__(self, df, n_neighbors = 5):
        #self.df = df # dataframe that preserves the feature names
        self.X, self.y = self.__get_values_and_labels__(self.df)
        self.n_neighbors = n_neighbors

    
    def __apply_filter__(self, X, y):

        N = len(self.df)
        clean_list = []

        knn = KNeighborsClassifier(n_neighbors=self.n_neighbors)
        knn.fit(self.X, self.y)
        for i in range(N):
            own_label = self.y[i]
            neighbors_idx = knn.kneighbors(self.X[i,:].reshape(1, -1), return_distance=False)
            neighbor_labels = self.y[neighbors_idx][0]
            values, counts = np.unique(neighbor_labels, return_counts=True)
            most_frequent_label = values[counts == counts.max()]
            clean_list.append([most_frequent_label == own_label][0][0])
        return clean_list

    def noise_index(self):
        self.clean_list = self.__apply_filter__(self, self.df)
        return self.clean_list
        
    def clean_samples(self):
        return self.df[self.df[self.clean_list]]

    def noisy_samples(self):
        return self.df[~self.df[self.clean_list]]

TypeError: module() takes at most 2 arguments (3 given)